In [1]:
# import the MAAP package to handle queries
from maap.maap import MAAP

import get_dem

# Choose bounding box

The bounding box should be a single string of four lat/lon, in the order:
[left  bottom  right top]

In [8]:

bbox = '-118.06817 34.22169 -118.05801 34.22822'  # Mt Wilson (near JPL) (small, ~8 sec)
# bbox = '-124.81360 32.44506 -113.75989 42.24498' # CA and NV (uses ~5-6 GB RAM, takes a few minutes to run)
# bbox = '-125.53514 25.42150  -87.06960 49.22105' # Western US (CA, WA, Chicago, New Orleans)
# bbox = '-125.41615 31.03621 -101.80916 49.17324' # Western US (CA -> CO)
# bbox =   '36.22293  6.42000   47.17150 18.70244'  # Africa
# bbox =    '6.26868 36.00380   18.57179 47.28139'  # Italy


## Test Run in the ADE

In [3]:
!conda env update -n base --file environment.yaml

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.7.4
    latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



certifi-2023.11.17   | 155 KB    |                                       |   0% 
sardem-0.11.3        | 31 KB     |                                       |   0% 

ca-certificates-2023 | 151 KB    |                                       |   0% 


openssl-3.2.0        | 2.7 MB    |                                       |   0% 

ca-certificates-2023 | 151 KB    | ###########8                          |  32% 

ca-certificates-2023 | 151 KB    | ##################################### | 100% 


certifi-2023.11.17   | 155 KB    | ###8                                  |  10% 


certifi-2023.11.17   | 155 KB    | ##############################5       |  82% 
sardem-0.11.3        | 31 KB  

In [9]:
# WARNING - only use for testing! Not DPS!
import importlib; importlib.reload(get_dem)
import os
from time import time

# make the output directory, if it does not already exist
os.makedirs("output")

# Build a DEM
start = time()

get_dem.get_dem(bbox, "output")

print(f"Time to make DEM: {time()-start} seconds")


[01/25 19:24:29] [INFO dem.py] Bounds: -118.06817 34.22169 -118.05801 34.22822
[01/25 19:24:29] [INFO cop_dem.py] Creating output/dem.tif
[01/25 19:24:29] [INFO cop_dem.py] Fetching remote tiles...
[01/25 19:24:29] [INFO cop_dem.py] Running GDAL command:
[01/25 19:24:29] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt output/dem.tif -of GTiff -ot Int16 -te -118.068169999999995 34.2216900000000024 -118.058009999999996 34.2282200000000003 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi


Creating output file that is 37P x 24L.
0...10...20...30...40...50...60...70...80...90...100 - done.
Time to make DEM: 8.123727798461914 seconds


## Launch jobs in DPS

In [20]:
# Register Algorithm from YAML file
# maap = MAAP()
# maap.register_algorithm_from_yaml_file("/projects/GET-DEM-v0.1.0.yml").text

'{"code": 200, "message": {"id": "1602b35737e86ad6910ca20eb1c8fd62f6033d50", "short_id": "1602b357", "created_at": "2024-01-26T05:14:44.000+00:00", "parent_ids": ["959120745fc1f2801bd38fd575045ec833729df2"], "title": "Registering algorithm: GET-DEM-v0.1.0", "message": "Registering algorithm: GET-DEM-v0.1.0", "author_name": "root", "author_email": "root@8e29c9ef876b", "authored_date": "2024-01-26T05:14:44.000+00:00", "committer_name": "root", "committer_email": "root@8e29c9ef876b", "committed_date": "2024-01-26T05:14:44.000+00:00", "trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/1602b35737e86ad6910ca20eb1c8fd62f6033d50", "stats": {"additions": 0, "deletions": 0, "total": 0}, "status": "running", "project_id": 3, "last_pipeline": {"id": 11668, "iid": 750, "project_id": 3, "sha": "1602b35737e86ad6910ca20eb1c8fd62f6033d50", "ref": "main", "status": "running", "source": "push", "created_at": "2024-01-26T05:14:45.204Z", "updated_at": "2024-01-26T

In [21]:
# Submit DPS Job
maap = MAAP()

jobs = []

for job_num in range(2):
    jobs.append(
        maap.submitJob(
            identifier="test1",
            algo_id="GET-DEM-v0.1.0",
            version="develop",
            username="niemoell",
            queue="maap-dps-worker-4vcpu-8gb",
            bbox=bbox  # use same bbox for all jobs
        )
    )


In [31]:
# Monitor the job status via JN.
# For a GUI, go to "View and Submit Jobs" via the Launcher
n_failed = 0
n_succeeded = 0
n_running = 0
n_except = 0
n_accepted = 0

flag = True

for j in jobs:
    try:
        j.retrieve_attributes()
    except:
        n_except += 1
        continue
    if j.status.lower() == 'failed':
        n_failed += 1
    elif j.status.lower() == 'succeeded':
        n_succeeded += 1
        if flag:
            # print(j)
            flag = False
    elif j.status.lower() == 'running':
        n_running += 1
    elif j.status.lower() == 'accepted':
        n_accepted += 1
    else:
        print(j.status)
    # print(j)

print("failed: ", n_failed)
print("accepted: ", n_accepted)
print("running: ", n_running)
print("succeeded: ", n_succeeded)
print("except: ", n_except)

failed:  0
accepted:  1
running:  1
succeeded:  0
except:  0


In [29]:
print(jobs[0])

{'job_id': '32ffb070-8ee3-436b-8b12-bce8943b6bed', 'status': 'Running', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': []}
